In [1]:
from nustar_pysolar import planning, io
import astropy.units as u

# Download the list of occultation periods from the MOC at Berkeley.

## Note that the occultation periods typically only are stored at Berkeley for the *future* and not for the past. So this is only really useful for observation planning.

In [2]:
fname = io.download_occultation_times(outdir='../data/')
print(fname)

../data/NUSTAR.2017_250.SHADOW_ANALYSIS.txt


# Download the NuSTAR TLE archive.

This contains every two-line element (TLE) that we've received for the whole mission. We'll expand on how to use this later.

The `times`, `line1`, and `line2` elements are now the TLE elements for each epoch.

In [3]:
tlefile = io.download_tle(outdir='../data')
print(tlefile)
times, line1, line2 = io.read_tle_file(tlefile)

../data/NuSTAR.tle


# Here is where we define the observing window that we want to use.

Note that tstart and tend must be in the future otherwise you won't find any occultation times and sunlight_periods will return an error.

In [7]:
tstart = '2017-09-11T00:00:00'
tend = '2017-09-15T00:00:00'
orbits = planning.sunlight_periods(fname, tstart, tend)

# We want to know how to orient NuSTAR for the Sun.

We can more or less pick any angle that we want. But this angle has to be specified a little in advance so that the NuSTAR SOC can plan the "slew in" maneuvers. Below puts DET0 in the top left corner (north-east with respect to RA/Dec coordinates).

### This is what you tell the SOC you want the "Sky PA angle" to be.

In [8]:
pa = planning.get_nustar_roll(tstart, 0)
print("NuSTAR Roll angle for Det0 in NE quadrant: {}".format(pa))

NuSTAR Roll angle for Det0 in NE quadrant: 23.324561616831126 deg


# Set up the offset you want to use here:

The first element is the direction +WEST of the center of the Sun, the second is the offset +NORTH of the center of the Sun.

If you want multiple pointing locations you can either specify an array of offsets or do this "by hand" below.

In [9]:
offset = [1100., -400.]*u.arcsec

# Loop over each orbit and correct the pointing for the same heliocentric pointing position.

Note that you may want to update the pointing for solar rotation. That's up to the user to decide and is not done here.

In [11]:
for ind, orbit in enumerate(orbits):
    midTime = (0.5*(orbit[1] - orbit[0]) + orbit[0])
    sky_pos = planning.get_sky_position(midTime, offset)
#    print("Orbit: {}".format(ind))
    print("Orbit start: {} Orbit end: {}".format(orbit[0].isoformat(), orbit[1].isoformat()))
    print("Aim Time: {}".format(midTime.isoformat()))
    print('Aim time: {} RA (deg): {} Dec (deg): {}'.format(midTime.isoformat(), sky_pos[0], sky_pos[1]))
    print("")

    


Orbit start: 2017-09-10T23:48:20 Orbit end: 2017-09-11T00:49:30
Aim Time: 2017-09-11T00:18:55
Aim time: 2017-09-11T00:18:55 RA (deg): 168.8444822500469 deg Dec (deg): 4.675665760939547 deg

Orbit start: 2017-09-11T01:25:00 Orbit end: 2017-09-11T02:26:20
Aim Time: 2017-09-11T01:55:40
Aim time: 2017-09-11T01:55:40 RA (deg): 168.90482172609111 deg Dec (deg): 4.650220901438387 deg

Orbit start: 2017-09-11T03:01:40 Orbit end: 2017-09-11T04:03:00
Aim Time: 2017-09-11T03:32:20
Aim time: 2017-09-11T03:32:20 RA (deg): 168.96510726129736 deg Dec (deg): 4.624791714579964 deg

Orbit start: 2017-09-11T04:38:20 Orbit end: 2017-09-11T05:39:40
Aim Time: 2017-09-11T05:09:00
Aim time: 2017-09-11T05:09:00 RA (deg): 169.02539085542193 deg Dec (deg): 4.599356312960442 deg

Orbit start: 2017-09-11T06:15:00 Orbit end: 2017-09-11T07:16:20
Aim Time: 2017-09-11T06:45:40
Aim time: 2017-09-11T06:45:40 RA (deg): 169.0856725353157 deg Dec (deg): 4.573914722834457 deg

Orbit start: 2017-09-11T07:51:40 Orbit end: 201

<Quantity [ 168.8811098 ,   4.69017351] deg>

# Sanity check: Reproduce the 2016-07-26 pointing.

Based on the IDL version of this code, when the aim time was 2016-07-26 19:53:15.00 the NP was 8.86 degrees and the RA and Dec were 126.04630 and 19.33532 deg, respectively for an offset of +1000 arcseconds West and +150 arcseconds North of the center of the Sun.

In [8]:
aim_time = '2016-07-26T19:53:15.00'
offset = [1000, 150]*u.arcsec
sky_pos = planning.get_sky_position(aim_time, offset)
print(sky_pos)

[ 126.04053869   19.33666449] deg


In [9]:
np = planning.get_nustar_roll(aim_time, 0)
print(np)

8.865884432089102 deg
